## Introduction
In this section, identifier (IDs) mapping is performed on an example transcriptomics data set, which was original annotated using HGNC symbols.
The dataset has been preprocessed already, for details see step 1 and 2 of the multi-omics workflow at: https://github.com/BiGCAT-UM/Transcriptomics_Metabolomics_Analysis/tree/master/transcriptomics_analysis .
We map the HGNC symbols to Entrez Gene and Ensembl IDs, since tools downstream of this step require different input formats for the IDs.

We use two tools for this mapping; first we use org.Hs.eg.db [doi:10.18129/B9.bioc.org.Hs.eg.db] and AnnotationDbi; second we use BridgeDb [doi:10.18129/B9.bioc.BridgeDbR].

## R environment setup


In [ ]:
# empty the R environment
rm (list = ls())
# check if libraries are already installed, otherwise install it
if(!requireNamespace("BiocManager", quietly = TRUE)) install.packages("BiocManager")
if(!"rstudioapi" %in% installed.packages()) BiocManager::install("rstudioapi")
if(!"org.Hs.eg.db" %in% installed.packages()) BiocManager::install("org.Hs.eg.db")  
if(!"AnnotationDbi" %in% installed.packages()) BiocManager::install("AnnotationDbi")
if(!"BridgeDbR" %in% installed.packages()) BiocManager::install("BridgeDbR")
if(!"dplyr" %in% installed.packages()) install.packages("dplyr")
if(!"rmarkdown" %in% installed.packages())install.packages("rmarkdown") 
if(!"data.table" %in% installed.packages())install.packages("data.table")
if(!"knitr" %in% installed.packages())install.packages("knitr")
# if(!"downloader" %in% installed.packages())install.packages("downloader")

#load installed libraries
suppressPackageStartupMessages({
  library(rstudioapi) # interface for interacting with RStudio IDE with R code.
  library(org.Hs.eg.db) #This is the organism annotation package ("org") for Homo sapiens ("Hs"), organized as an AnnotationDbi   package ("db"), using Entrez Gene IDs ("eg") as primary key.
  library(AnnotationDbi) # for connecting and querying annotation databases
  library(BridgeDbR)
  library(dplyr)
  library(rmarkdown)
  library(data.table)
  library(knitr)
  # require(downloader)
})

# set your working environment to the location where your current source file is saved into.
setwd(dirname(rstudioapi::getSourceEditorContext()$path))


## Importing dataset
The data will be read for the disease on two biopsy locations


In [ ]:
#we have four datasets, two different disorders and two different biopsy locations
#Reading the files for CD disorder
(filenames <- list.files("data", pattern = "CD", full.names = TRUE)) #Printing the file names
dataset_CD <- lapply(filenames, read.delim) #Reading the files

#filter out  unused columns, we select geneSymbol, log2FC and pvalue and
#merge two dataset of two locations into one datafile
dataset_CD <- merge(dataset_CD[[1]] %>% select("X", "FoldChange", "padj"), #Merging and sub-setting the columns required
                    dataset_CD[[2]] %>% select("X", "FoldChange", "padj"), 
                    by = "X", all = TRUE)

#Reading the files for UC disorder
(filenames <- list.files("data", pattern = "UC", full.names = TRUE)) #Printing the file names
dataset_UC <- lapply(filenames, read.delim) #Reading the files

#filter out  unused columns, we select geneSymbol, log2FC and pvalue and
#merge two dataset of two locations into one datafile per disorder
dataset_UC <- merge(dataset_UC[[1]] %>% select("X", "FoldChange", "padj"), #Merging and sub-setting the columns required
                    dataset_UC[[2]] %>% select("X", "FoldChange", "padj"), 
                    by = "X", all = TRUE)

#change column names
colnames(dataset_CD) <- colnames(dataset_UC) <- c("GeneSymbol", "log2FC_ileum", "pvalue_ileum", "log2FC_rectum", "pvalue_rectum")

rm(list = setdiff(ls(), c("dataset_UC", "dataset_CD"))) # removing variables that are not required



## Converting hgnc gene symbols to the corresponding Entrez (NCBI) gene IDs (org.Hs.eg.db)


In [ ]:
#converting gene symbols to entrez ID since these are required for the enrichR function
hs <- org.Hs.eg.db #This object is a simple mapping of Entrez Gene identifier

# check if all the gene symbols are the same in both datasets
all(dataset_CD$GeneSymbol == dataset_UC$GeneSymbol) 
##same gene symbols, so mapping based on one of the files 
entrezID <- AnnotationDbi::select(hs, keys = dataset_CD$GeneSymbol, 
            columns = c("ENTREZID", "SYMBOL"), 
            keytype = "SYMBOL")

# checking the one-to-multiple mappings
if(!all(table(entrezID$SYMBOL) == 1)) print ("There are one-to-multiple mappings")
#store one-to-multiple mapping info
entrezID_doubles_Hs <- length(table(entrezID$SYMBOL) [table(entrezID$SYMBOL) > 1])
##run the two lines below if you want to check which genes have multiple Entrez (NCBI) gene IDs
# entrezID_doubles_Hs <- names(table(entrezID$SYMBOL)[table(entrezID$SYMBOL) > 1])
# entrezID [entrezID$SYMBOL %in% entrezID_doubles_Hs, ]

#filter out double identifiers because there are one-to-many relationship
entrezID <- entrezID %>% distinct(entrezID$SYMBOL, .keep_all = TRUE)

# add entrezIDs for each gene symbol in the dataset
dataset_CD$ENTREZ.ID_org.Hs <- entrezID$ENTREZID [match(dataset_CD$GeneSymbol, entrezID$SYMBOL)] 
dataset_UC$ENTREZ.ID_org.Hs <- entrezID$ENTREZID [match(dataset_UC$GeneSymbol, entrezID$SYMBOL)] 

rm(list = setdiff(ls(), c("dataset_UC", "dataset_CD", "entrezID_doubles_Hs", "hs"))) # removing variables that are not required



## Converting hgnc gene symbols to the corresponding Ensembl IDs (org.Hs.eg.db)


In [ ]:
#converting gene symbols to Ensembl ID since these are required for the Cytoscape multiomics visualization
ensemblID <- AnnotationDbi::select(hs, keys = dataset_CD$GeneSymbol, 
            columns = c("ENSEMBL", "SYMBOL"), 
            keytype = "SYMBOL")

# checking the one-to-multiple mappings
if(!all(table(ensemblID$SYMBOL) == 1)) print ("There are one-to-multiple mappings")
#store one-to-multiple mapping info
ensemblID_doubles_Hs <- length(table(ensemblID$SYMBOL)[table(ensemblID$SYMBOL) > 1])
##run the two lines below if you want to check which genes have multiple Ensembl IDs
# ensemblID_doubles_Hs <- names(table(ensemblID$SYMBOL)[table(ensemblID$SYMBOL) > 1])
# ensemblID %>% filter(SYMBOL %in% ensemblID_doubles_Hs) %>% arrange(SYMBOL)

#filter out double identifiers because there are one-to-many relationship
ensemblID <- ensemblID %>% distinct(ensemblID$SYMBOL, .keep_all = TRUE)
# add entrezIDs for each gene symbol in the dataset
dataset_CD$Ensembl.ID_org.Hs <- ensemblID$ENSEMBL [match(dataset_CD$GeneSymbol, ensemblID$SYMBOL)] 
dataset_UC$Ensembl.ID_org.Hs <- ensemblID$ENSEMBL [match(dataset_UC$GeneSymbol, ensemblID$SYMBOL)] 

rm(list = setdiff(ls(), c("dataset_UC", "dataset_CD", "entrezID_doubles_Hs", "ensemblID_doubles_Hs"))) # removing variables that are not required



## Converting hgnc gene symbols to the corresponding Entrez (NCBI) gene IDs (BridgeDb)


In [ ]:
#Download and load the human derby database and converting gene symbols to entrez ID since these are required for the enrichR function
# fileUrl <- ""
# require(downloader)
# download(fileUrl, "data/Hs_Derby_Ensembl_105.bridge", mode = "wb")
mapper <- loadDatabase("D:/bridgeDb/BridgeDbDemoBioSB2022/scripts/1-identifier_mapping_transcriptomics/data/Hs_Derby_Ensembl_105.bridge")

## Obtain the System codes for the databases HGNC (source database of dataset) and Entrez (NCBI) (intended output database)
code_mappingFrom <- getSystemCode("HGNC")
code_mappingTo <- getSystemCode("Entrez Gene")

## Create a data frame with the mappings and the correct SystemCode
input <- data.frame(source = rep(code_mappingFrom, length(dataset_CD$GeneSymbol)),
                     identifier = dataset_CD$GeneSymbol)
#Obtain all mappings from HGNC to Entrez (NCBI)
entrezID <- maps(mapper = mapper, input, target = code_mappingTo) 

# checking the one-to-multiple mappings
if(!all(table(entrezID$identifier) == 1)) print ("There are one-to-multiple mappings")
#store one-to-multiple mapping info
entrezID_doubles_BridgeDb <- length(table(entrezID$identifier) [table(entrezID$identifier) > 1])
##run the two lines below if you want to check which genes have multiple Entrez (NCBI) gene IDs
# entrezID_doubles_BridgeDb <- names(table(entrezID$identifier)[table(entrezID$identifier) > 1])
# entrezID [entrezID$identifier %in% entrezID_doubles_BridgeDb, ]

#filter out double identifiers because there are one-to-many relationship
entrezID <- entrezID %>% distinct(entrezID$identifier, .keep_all = TRUE)

# add entrezIDs for each gene symbol in the dataset
dataset_CD$ENTREZ.ID_BridgeDb <- entrezID$mapping [match(dataset_CD$GeneSymbol, entrezID$identifier)] 
dataset_UC$ENTREZ.ID_BridgeDb <- entrezID$mapping [match(dataset_UC$GeneSymbol, entrezID$identifier)] 

rm(list = setdiff(ls(), c("dataset_UC", "dataset_CD", "entrezID_doubles_Hs", "ensemblID_doubles_Hs", "mapper", "input", "entrezID_doubles_BridgeDb"))) # removing variables that are not required



## Converting hgnc gene symbols to the corresponding Ensembl IDs (BridgeDb)


In [ ]:
#converting gene symbols to Ensembl ID since these are required for the Cytoscape multiomics visualization
## Obtain the System codes for Ensembl (intended output database)
code_mappingTo <- getSystemCode("Ensembl")
#Obtain all mappings from HGNC to Ensembl
ensemblID <- maps(mapper = mapper, input, target = code_mappingTo) 

# checking the one-to-multiple mappings
if(!all(table(ensemblID$identifier) == 1)) print ("There are one-to-multiple mappings")
#store one-to-multiple mapping info
ensemblID_doubles_BridgeDb <- length(table(ensemblID$identifier) [table(ensemblID$identifier) > 1])
##run the two lines below if you want to check which genes have multiple Ensembl ID
# ensemblID_doubles_BridgeDb <- names(table(ensemblID$identifier)[table(ensemblID$identifier) > 1])
# ensemblID [ensemblID$identifier %in% ensemblID_doubles_BridgeDb, ] 

#filter out double identifiers because there are one-to-many relationship
ensemblID <- ensemblID %>% distinct(ensemblID$identifier, .keep_all = TRUE)

# add ensemblIDs for each gene symbol in the dataset
dataset_CD$Ensembl.ID_BridgeDb <- ensemblID$mapping [match(dataset_CD$GeneSymbol, ensemblID$identifier)] 
dataset_UC$Ensembl.ID_BridgeDb <- ensemblID$mapping [match(dataset_UC$GeneSymbol, ensemblID$identifier)] 

rm(list = setdiff(ls(), c("dataset_UC", "dataset_CD", "entrezID_doubles_Hs", "ensemblID_doubles_Hs", "mapper", "input", "entrezID_doubles_BridgeDb", "ensemblID_doubles_BridgeDb"))) # removing variables that are not required


## Using BridgeDb for secondary to primary mapping of hgnc gene symbols

### a. mapping the hgnc symbols to hgnc IDs
HGNC recycles the hgnc gene symbols which means there are identifiers (symbols) that are secondary for a certain entity (e.g the primary hgnc symbol is FYN and the secondary hgnc symbol is SLK), while the same identifiers might be used as a primary symbol for another entity (SLK is a primary symbol for another gene). 
This makes the mapping based on hgnc gene symbols challenging. 
`Here We assumed when a hgnc gene symbol has a hgnc ID, it is primary and we only map those without a hgnc ID.`


In [ ]:
#converting gene symbols to hgnc ID 
## Obtain the System codes for HGNC Accession number (intended output database)
code_mappingTo <- getSystemCode("HGNC Accession number")
#Obtain all mappings from HGNC to HGNC Accession number
hgncID <- maps(mapper = mapper, input, target = "Hac") 

# checking the one-to-multiple mappings
if(!all(table(hgncID$identifier) == 1)) {print ("There are one-to-multiple mappings.")} else  print ("There is no one-to-multiple mapping.")

# add HGNC id for each gene symbol in the dataset
dataset_CD$HGNC.ID_BridgeDb <- hgncID$mapping[match(dataset_CD$GeneSymbol, hgncID$identifier)]
dataset_UC$HGNC.ID_BridgeDb <- hgncID$mapping[match(dataset_UC$GeneSymbol, hgncID$identifier)]

rm(list = setdiff(ls(), c("dataset_UC", "dataset_CD", "entrezID_doubles_Hs", "ensemblID_doubles_Hs", "entrezID_doubles_BridgeDb", "ensemblID_doubles_BridgeDb"))) # removing variables that are not required


### b. mapping secondary hgnc symbols to primary hgnc symbols
Mapping hgnc symbols without a HGNC id mapped


In [ ]:
# loading the HGNC secondary id derby database
mapper <- loadDatabase("D:/bridgeDb/BridgeDbDemoBioSB2022/scripts/1-identifier_mapping_transcriptomics/data/hgnc_primaryToSecondaryIDs.bridge")

## Obtain the System codes for the databases HGNC 
code_mapping <- getSystemCode("HGNC")

#Subset hgnc gene symbols with no hgnc ID
input <- dataset_CD$GeneSymbol [is.na(dataset_CD$HGNC.ID_BridgeDb)] 
## Create a data frame with the mappings and the correct SystemCode
input <- data.frame(source = rep(code_mapping, length(input)),
                    identifier = input)
#converting secondary gene symbols to primary gene symbols 
hgnc <- maps(mapper = mapper, input, target = code_mapping) %>% 
    filter(isPrimary == "T") # Keeping only rows where the mapping is annotated as primary id (defined in BridgeDb java library when creating the derby database)

# checking the one-to-multiple mappings
if(!all(table(hgnc$identifier) == 1)) print ("There are one-to-multiple mappings")

#store one-to-multiple mapping info
hgnc_doubles_PriID_BridgeDb <- length(table(hgnc$identifier) [table(hgnc$identifier) > 1])
##run the two lines below if you want to check which genes have multiple mapping
# hgnc_doubles_PriID_BridgeDb <- names(table(hgnc$identifier)[table(hgnc$identifier) > 1])
# hgnc [hgnc$identifier %in% hgnc_doubles_PriID_BridgeDb, ] 


Checking the reason that the secondary symbol mapped to multiple primary symbols



In [ ]:
hgnc [hgnc$identifier == "AGPAT9", ]



In this example where we manually checked:
GPAT3 is the Previous symbol (this field displays any symbols that were previously HGNC-approved nomenclature.)
LPCAT1 is the Alias symbol (Alternative symbols that have been used to refer to the gene. Aliases may be from literature, from other databases or may be added to represent membership of a gene group.)


In [ ]:
hgnc [hgnc$identifier == "U3", ]



This is an example of using the same alias for 5 different hgnc genes

The issue of one-to-multiple mappings could not be easily fixed (needs manual check) and emphasizes on the importance of using unique, persistent, resolvable identifiers. 

Some of the primary symbols are already presented in the dataset (used as primary symbol for another gene). This shows another limitation of using gene symbols as identifiers, and to avoid the duplicated gene symbols we only mapped those symbols that are not presented in the dataset. 


In [ ]:
if (!all(!hgnc$mapping %in% dataset_CD$GeneSymbol))
    hgnc <- hgnc [!hgnc$mapping %in% dataset_CD$GeneSymbol,] 
    
#filter out double gene symbols
hgnc <- hgnc %>% distinct(hgnc$identifier, .keep_all = TRUE)

# add primary (current) hgnc gene symbol each gene symbol in the dataset
dataset_CD$Current_GeneSymbol <- hgnc$mapping[match(dataset_CD$GeneSymbol, hgnc$identifier)]
dataset_CD$Current_GeneSymbol [is.na(dataset_CD$Current_GeneSymbol)] = dataset_CD$GeneSymbol [is.na(dataset_CD$Current_GeneSymbol)]

dataset_UC$Current_GeneSymbol <- hgnc$mapping[match(dataset_UC$GeneSymbol, hgnc$identifier)]
dataset_UC$Current_GeneSymbol [is.na(dataset_UC$Current_GeneSymbol)] = dataset_UC$GeneSymbol [is.na(dataset_UC$Current_GeneSymbol)]

rm(list = setdiff(ls(), c("dataset_UC", "dataset_CD", "entrezID_doubles_Hs", "ensemblID_doubles_Hs", "entrezID_doubles_BridgeDb", "ensemblID_doubles_BridgeDb", "hgnc_doubles_PriID_BridgeDb"))) # removing variables that are not required



## Converting `primary` hgnc gene symbols to the corresponding Entrez (NCBI) gene IDs (BridgeDb)


In [ ]:
#load the human derby database and converting gene symbols to entrez ID since these are required for the enrichR function
mapper <- loadDatabase("D:/bridgeDb/BridgeDbDemoBioSB2022/scripts/1-identifier_mapping_transcriptomics/data/Hs_Derby_Ensembl_105.bridge")

# Obtain the System codes for the databases HGNC (source database of dataset) and Entrez (NCBI) (intended output database)
code_mappingFrom <- getSystemCode("HGNC")
code_mappingTo <- getSystemCode("Entrez Gene")

# Create a data frame with the mappings and the correct SystemCode
input <- data.frame(source = rep(code_mappingFrom, length(dataset_CD$Current_GeneSymbol)),
                    identifier = dataset_CD$Current_GeneSymbol)

#Obtain all mappings from HGNC to Entrez (NCBI)
entrezID <- maps(mapper = mapper, input, target = code_mappingTo) 

# checking the one-to-multiple mappings
if(!all(table(entrezID$identifier) == 1)) print ("There are one-to-multiple mappings")
#store one-to-multiple mapping info
entrezID_doubles_PriID_BridgeDb <- length(table(entrezID$identifier) [table(entrezID$identifier) > 1])
##run the two lines below if you want to check which genes have multiple Entrez (NCBI) gene IDs
# entrezID_doubles_PriID_BridgeDb <- names(table(entrezID$identifier)[table(entrezID$identifier) > 1])
# entrezID [entrezID$identifier %in% entrezID_doubles_PriID_BridgeDb, ] 

#filter out double identifiers because there are one-to-many relationship
entrezID <- entrezID %>% distinct(entrezID$identifier, .keep_all = TRUE)

# add entrezIDs for each gene symbol in the dataset
dataset_CD$ENTREZ.ID_PriID_BridgeDb <- entrezID$mapping [match(dataset_CD$Current_GeneSymbol, entrezID$identifier)] 
dataset_UC$ENTREZ.ID_PriID_BridgeDb <- entrezID$mapping [match(dataset_UC$Current_GeneSymbol, entrezID$identifier)] 

rm(list = setdiff(ls(), c("dataset_UC", "dataset_CD", "entrezID_doubles_Hs", "ensemblID_doubles_Hs", "entrezID_doubles_BridgeDb", "ensemblID_doubles_BridgeDb", "hgnc_doubles_PriID_BridgeDb", "entrezID_doubles_PriID_BridgeDb", "input", "mapper"))) # removing variables that are not required



## Converting `primary` hgnc gene symbols to the corresponding Ensembl IDs (BridgeDb)


In [ ]:
#converting gene symbols to Ensembl ID since these are required for the Cytoscape multiomics visualization
## Obtain the System codes for Ensembl (intended output database)
code_mappingTo <- getSystemCode("Ensembl")
#Obtain all mappings from HGNC to Ensembl
ensemblID <- maps(mapper = mapper, input, target = code_mappingTo) 

# checking the one-to-multiple mappings
if(!all(table(ensemblID$identifier) == 1)) print ("There are one-to-multiple mappings")
#store one-to-multiple mapping info
ensemblID_doubles_PriID_BridgeDb <- length(table(ensemblID$identifier) [table(ensemblID$identifier) > 1])
##run the two lines below if you want to check which genes have multiple Ensembl ID
# ensemblID_doubles_PriID_BridgeDb <- names(table(ensemblID$identifier)[table(ensemblID$identifier) > 1])
# ensemblID [ensemblID$identifier %in% ensemblID_doubles_PriID_BridgeDb, ]

#filter out double identifiers because there are one-to-many relationship
ensemblID <- ensemblID %>% distinct(ensemblID$identifier, .keep_all = TRUE)

# add ensemblIDs for each gene symbol in the dataset
dataset_CD$Ensembl.ID_PriID_BridgeDb <- ensemblID$mapping [match(dataset_CD$Current_GeneSymbol, ensemblID$identifier)] 
dataset_UC$Ensembl.ID_PriID_BridgeDb <- ensemblID$mapping [match(dataset_UC$Current_GeneSymbol, ensemblID$identifier)] 

rm(list = setdiff(ls(), c("dataset_UC", "dataset_CD", "entrezID_doubles_Hs", "ensemblID_doubles_Hs", "entrezID_doubles_BridgeDb", "ensemblID_doubles_BridgeDb", "hgnc_doubles_PriID_BridgeDb", "entrezID_doubles_PriID_BridgeDb", "ensemblID_doubles_PriID_BridgeDb"))) # removing variables that are not required



##Mapping stats:


In [ ]:
##HGNC_originalData
paste0("The total number of HGNC Symbol in the transcriptomics dataset is: ", nrow(dataset_CD))

MappingStats <- data.table(`  ` =  c("The total number of unique Entrez IDs",
                                     "The total number of missing mappings for HGNC Symbol to Entrez IDs",
                                     "The total number of one-to-many mappings for Entrez IDs",
                                     "The total number of unique Ensembl IDs",
                                     "The total number of missing mappings for HGNC Symbol to Ensembl IDs",
                                     "The total number of one-to-many mappings for Ensembl IDs"),
                           org.Hs = c(sum(!is.na(dataset_CD$ENTREZ.ID_org.Hs)), 
                                      sum(is.na(dataset_CD$ENTREZ.ID_org.Hs)),
                                      entrezID_doubles_Hs,
                                      sum(!is.na(dataset_CD$Ensembl.ID_org.Hs)), 
                                      sum(is.na(dataset_CD$Ensembl.ID_org.Hs)), 
                                      ensemblID_doubles_Hs),
                           BridgeDb = c(sum(!is.na(dataset_CD$ENTREZ.ID_BridgeDb)),
                                        sum(is.na(dataset_CD$ENTREZ.ID_BridgeDb)), 
                                        entrezID_doubles_BridgeDb,
                                        sum(!is.na(dataset_CD$Ensembl.ID_BridgeDb)), 
                                        sum(is.na(dataset_CD$Ensembl.ID_BridgeDb)),
                                        ensemblID_doubles_BridgeDb),
                           PrimaryID_BridgeDb = c(sum(!is.na(dataset_CD$ENTREZ.ID_PriID_BridgeDb)),
                                                  sum(is.na(dataset_CD$ENTREZ.ID_PriID_BridgeDb)),
                                                  entrezID_doubles_PriID_BridgeDb,
                                                  sum(!is.na(dataset_CD$Ensembl.ID_PriID_BridgeDb)), 
                                                  sum(is.na(dataset_CD$Ensembl.ID_PriID_BridgeDb)),
                                                  ensemblID_doubles_PriID_BridgeDb))

kable(MappingStats)
##Citation org.Hs:
citation("org.Hs.eg.db")

##Citation BridgeDb:
citation("BridgeDbR")



##Save data, print session info and remove large datasets:


In [ ]:
##Save data: exporting results to the file
#CD
write.table(dataset_CD, file = "results/IDMapping_CD", 
            sep = "\t" , quote = FALSE, row.names = FALSE)

#UC
write.table(dataset_UC, file = "results/IDMapping_UC", 
            sep = "\t" , quote = FALSE, row.names = FALSE)

##Print session info:
sessionInfo()

##Remove data objects which are not needed for further processing:
rm(list = setdiff(ls(), c("dataset_CD", "dataset_UC")))


In [ ]:
#Jupyter Notebook file
# if(!"devtools" %in% installed.packages()) BiocManager::install("devtools")
# if(!"rmd2jupyter" %in% installed.packages()) devtools::install_github("mkearney/rmd2jupyter", force=TRUE)
# library(devtools)
# library(rmd2jupyter)
# setwd(dirname(rstudioapi::getSourceEditorContext()$path))
# rmd2jupyter("identifier_mapping_transcriptomics.Rmd")
